In [1]:
import pandas as pd
import numpy as np
import nltk
from nltk.stem import SnowballStemmer
from nltk.tokenize import word_tokenize, sent_tokenize
from nltk.corpus import stopwords
import re

encoding = "ISO-8859-1"
nltk.download('stopwords')
nltk.download('punkt')
nltk.download('averaged_perceptron_tagger')

[nltk_data] Downloading package stopwords to /home/eugene/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to /home/eugene/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /home/eugene/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!


True

In [2]:
stemmer = SnowballStemmer('english')
stop_words = set(stopwords.words('english'))
stop_words.add('xby')

typos = {
    'ennox': 'lennox',
    'garde': 'garden',
    'unsulation': 'insulation',
    'carboy': 'carbon',
    'grqss': 'grass',
    'spaonges': 'sponges',
    'fen': 'fern',
    'free shipping': 'null'
}


def replaces(s):
    if isinstance(s, str):
        s = s.lower()
        if s in typos:
            return typos[s]
        
        s = re.sub(r"(\w)[\.:]?([A-Z])", r"\1 \2", s)
    
        s = s.replace(" x "," xby ")
        s = s.replace("*"," xby ")
        s = s.replace(" by "," xby")
        s = s.replace("x0"," xby 0")
        s = s.replace("x1"," xby 1")
        s = s.replace("x2"," xby 2")
        s = s.replace("x3"," xby 3")
        s = s.replace("x4"," xby 4")
        s = s.replace("x5"," xby 5")
        s = s.replace("x6"," xby 6")
        s = s.replace("x7"," xby 7")
        s = s.replace("x8"," xby 8")
        s = s.replace("x9"," xby 9")
        s = s.replace("0x","0 xby ")
        s = s.replace("1x","1 xby ")
        s = s.replace("2x","2 xby ")
        s = s.replace("3x","3 xby ")
        s = s.replace("4x","4 xby ")
        s = s.replace("5x","5 xby ")
        s = s.replace("6x","6 xby ")
        s = s.replace("7x","7 xby ")
        s = s.replace("8x","8 xby ")
        s = s.replace("9x","9 xby ")
        
        s = re.sub(r"([0-9])( *)\.( *)([0-9])", r"\1.\4", s)
        s = re.sub(r"([0-9]+)( *)(inches|inch|in|')\.?", r"\1 in. ", s)
        s = re.sub(r"([0-9]+)( *)(foot|feet|ft|'')\.?", r"\1 ft. ", s)
        s = re.sub(r"([0-9]+)( *)(pounds|pound|lbs|lb)\.?", r"\1 lb. ", s)
        s = re.sub(r"([0-9]+)( *)(square|sq) ?\.?(feet|foot|ft)\.?", r"\1sq.ft. ", s)
        s = re.sub(r"([0-9]+)( *)(cubic|cu) ?\.?(feet|foot|ft)\.?", r"\1cu.ft. ", s)
        s = re.sub(r"([0-9]+)( *)(gallons|gallon|gal)\.?", r"\1gal. ", s)
        s = re.sub(r"([0-9]+)( *)(ounces|ounce|oz)\.?", r"\1oz. ", s)
        s = re.sub(r"([0-9]+)( *)(centimeters|cm)\.?", r"\1cm. ", s)
        s = re.sub(r"([0-9]+)( *)(milimeters|mm)\.?", r"\1mm. ", s)
        s = s.replace("°"," degrees ")
        s = re.sub(r"([0-9]+)( *)(degrees|degree)\.?", r"\1deg. ", s)
        s = s.replace(" v "," volts ")
        s = re.sub(r"([0-9]+)( *)(volts|volt)\.?", r"\1volt. ", s)
        s = re.sub(r"([0-9]+)( *)(watts|watt)\.?", r"\1watt. ", s)
        s = re.sub(r"([0-9]+)( *)(amperes|ampere|amps|amp)\.?", r"\1amp. ", s)
        
        s = s.replace("toliet","toilet")
        s = s.replace("airconditioner","air conditioner")
        s = s.replace("vinal","vinyl")
        s = s.replace("vynal","vinyl")
        s = s.replace("skill","skil")
        s = s.replace("snowbl","snow bl")
        s = s.replace("plexigla","plexi gla")
        s = s.replace("rustoleum","rust-oleum")
        s = s.replace("whirpool","whirlpool")
        s = s.replace("whirlpoolga", "whirlpool ga")
        s = s.replace("whirlpoolstainless","whirlpool stainless")
        s = s.replace("whirlpoolstainless","whirlpool stainless")
        s = s.replace('&amp;', '')
        s = s.replace("moulding","molding")

        s = s.replace("  "," ")
        
        s = s.replace("/", " ")
        s = s.replace("-", " ")
        return " ".join([re.sub('[^A-Za-z0-9./]', ' ', word)
                         for word in s.split()
                         if word.isdigit() or len(word)>1])
    else:
        return "null"

@np.vectorize
def stem_sentence(s): 
    s = replaces(s)   
    return str(" ".join([stemmer.stem(word)
                         for word in s.split()
                         if word not in stop_words]))


In [3]:
encoding = "ISO-8859-1"

products = pd.read_csv('zip/product_descriptions.csv.gz', encoding=encoding, compression='gzip',
                       index_col=['product_uid'])
products['orig_descr'] = products['product_description'].astype(str)
products['descr'] = stem_sentence(products['orig_descr'].astype(str))

In [4]:
products = products.drop(columns=['product_description'])
products.to_csv('zip/descr3.csv.gz', compression='gzip')

In [5]:
train = pd.read_csv('zip/train.csv.gz', index_col=['id'], compression='gzip', encoding=encoding)
train.head()

,product_uid,product_title,search_term,relevance
id,,,,
2,100001,Simpson Strong-Tie 12-Gauge Angle,angle bracket,3.00
3,100001,Simpson Strong-Tie 12-Gauge Angle,l bracket,2.50
9,100002,BEHR Premium Textured DeckOver 1-gal. #SC-141 ...,deck over,3.00
16,100005,Delta Vero 1-Handle Shower Only Faucet Trim Ki...,rain shower head,2.33
17,100005,Delta Vero 1-Handle Shower Only Faucet Trim Ki...,shower only faucet,2.67


In [6]:
train['orig_search_term'] = train['search_term'].astype(str)
train['search_term'] = stem_sentence(train['orig_search_term'].astype(str) )
train.loc[train['search_term'] == '', 'search_term'] = 'null'
train['search_term'] = train['search_term'].astype(str)

train['orig_product_title'] = train['product_title'].astype(str)
train['product_title'] = stem_sentence(train['product_title'].astype(str) )
train['product_title'] = train['product_title'].astype(str)

train['orig_descr'] = train.product_uid.map(products['orig_descr'])
train['descr'] = train.product_uid.map(products['descr'])

train.head()

,product_uid,product_title,search_term,relevance,orig_search_term,orig_product_title,orig_descr,descr
id,,,,,,,,
2,100001,simpson strong tie 12 gaug angl,angl bracket,3.00,angle bracket,Simpson Strong-Tie 12-Gauge Angle,"Not only do angles make joints stronger, they ...",angl make joint stronger also provid consist s...
3,100001,simpson strong tie 12 gaug angl,bracket,2.50,l bracket,Simpson Strong-Tie 12-Gauge Angle,"Not only do angles make joints stronger, they ...",angl make joint stronger also provid consist s...
9,100002,behr premium textur deckov 1 gal. sc 141 tugbo...,deck,3.00,deck over,BEHR Premium Textured DeckOver 1-gal. #SC-141 ...,BEHR Premium Textured DECKOVER is an innovativ...,behr premium textur deckov innov solid color c...
16,100005,delta vero 1 handl shower faucet trim kit chro...,rain shower head,2.33,rain shower head,Delta Vero 1-Handle Shower Only Faucet Trim Ki...,Update your bathroom with the Delta Vero Singl...,updat bathroom delta vero singl handl shower f...
17,100005,delta vero 1 handl shower faucet trim kit chro...,shower faucet,2.67,shower only faucet,Delta Vero 1-Handle Shower Only Faucet Trim Ki...,Update your bathroom with the Delta Vero Singl...,updat bathroom delta vero singl handl shower f...


In [7]:
train.to_csv('zip/train8.csv.gz', compression='gzip')

In [8]:
from fuzzywuzzy import fuzz


@np.vectorize
def word_match_count(search_term, text):
    count = 0
    search_term = search_term.split()
    text = text.split()
    if len(search_term) == 0:
        return 0
    for word in set(search_term):
        count += 1 if text.count(word) > 0 else 0
    return count / len(search_term)


@np.vectorize
def word_match_std(search_term, text):
    indexes = []
    search_term = search_term.split()
    text = text.split()    
    n = len(text)
    if len(search_term) == 0:
        return 0
    for word in set(search_term):
        if word in text:
            indexes.append(text.index(word))
    if len(indexes) > 0:
        return np.var(indexes)
    else:
        return 0

    
@np.vectorize
def match_last_word(search_term, text):
    last_word = search_term.split()[-1]
    if text.find(last_word) > -1:
        return 1
    else:
        return 0


@np.vectorize    
def word_match_index(search_term, text):
    count = 0
    search_term = search_term.split()
    text = text.split()
    if len(search_term) == 0:
        return 0
    min_ind = len(text)
    for word in set(search_term):
        if word in text:
            min_ind = min(min_ind, text.index(word))
    return min_ind / len(text) 


@np.vectorize 
def match_word_n_pos(text, search_term):
    text = nltk.pos_tag(replaces(text).split())
    search_pos = nltk.pos_tag(replaces(search_term).split())
    text = ["%s/%s" % (word[0].lower(), word[1]) for word in text]
    search_pos = ["%s/%s" % (word[0].lower(), word[1]) for word in search_pos]
    
    if len(search_pos) == 0:
        return 0
    matches = 0
    for word in set(text):
        if word in search_pos:
            matches += 1
    return matches / len(search_pos)


@np.vectorize
def match_first_word(search_term, text):
    last_word = search_term.split()[0]
    if text.find(last_word) > -1:
        return 1
    else:
        return 0

    
@np.vectorize
def match_numbers(search_term, text):
    search_term_numbers = set(re.findall(r'\d+', search_term))
    len_ = len(search_term_numbers)
    if len_ == 0:
        return 0
    text_numbers = set(re.findall(r'\d+', text))
    match = len([1 for i in search_term_numbers if i in text_numbers])
    return match / len_


@np.vectorize
def fuzzy_match(search_term, field):
    if search_term == 'null':
        return 0
    return fuzz.token_set_ratio(search_term, field)


@np.vectorize
def fuzzy_match_partial(search_term, field):
    if search_term == 'null':
        return 0
    return fuzz.partial_token_set_ratio(search_term, field)

In [9]:
attrs = pd.read_csv('zip/attributes.csv.gz', compression='gzip', encoding=encoding)
brand = attrs[attrs['name'] == 'MFG Brand Name']
brand.index = brand['product_uid']
brand['value'] = stem_sentence(brand['value'].astype(str))
brand['value'] = brand['value'].astype(str)
brand.head()

/home/eugene/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  after removing the cwd from sys.path.
/home/eugene/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """


,product_uid,name,value
product_uid,,,
100001.0,100001.0,MFG Brand Name,simpson strong tie
100002.0,100002.0,MFG Brand Name,behr premium textur deckov
100003.0,100003.0,MFG Brand Name,sterl
100004.0,100004.0,MFG Brand Name,grape solar
100005.0,100005.0,MFG Brand Name,delta


In [10]:
brand.drop(columns=['product_uid', 'name'])
brand.to_csv('zip/brands8.csv.gz', compression='gzip')

In [11]:
%%time

for col in ['product_title', 'descr']:
    train['match_' + col] = word_match_count(train['search_term'], train[col])
    
train['len_search_term_words'] = train['search_term'].apply(lambda x: len(x.split()))
train['len_search_term_letters'] = train['search_term'].str.len()

train['match_pos_title'] = match_word_n_pos(train['orig_product_title'], train['orig_search_term'])
train['match_pos_descr'] = match_word_n_pos(train['orig_descr'], train['orig_search_term'])

train['prod_title_ind'] = word_match_index(train['search_term'], train['product_title'])
train['descr_ind'] = word_match_index(train['search_term'], train['descr'])

train['words_std_title'] = word_match_std(train['search_term'], train['product_title'])
train['words_std_descr'] = word_match_std(train['search_term'], train['descr'])

train['len_title'] = train['product_title'].apply(lambda x: len(x.split()))
train['len_descr'] = train['descr'].apply(lambda x: len(x.split()))

train['last_word_title'] = match_last_word(train['search_term'], train['product_title'])  
train['last_word_descr'] = match_last_word(train['search_term'], train['descr']) 

train['match_orig_title'] = word_match_count(train['orig_search_term'], train['orig_product_title'])
train['match_orig_descr'] = word_match_count(train['orig_search_term'], train['orig_descr'])

train['fuzzy_title'] = fuzzy_match(train['search_term'], train['product_title'])
train['fuzzy_descr'] = fuzzy_match(train['search_term'], train['descr'])

train['fuzzy_orig_title'] = fuzzy_match(train['orig_search_term'], train['orig_product_title'])
train['fuzzy_orig_descr'] = fuzzy_match(train['orig_search_term'], train['orig_descr'])

train['part_fuzzy_title'] = fuzzy_match_partial(train['search_term'], train['product_title'])
train['part_fuzzy_descr'] = fuzzy_match_partial(train['search_term'], train['descr'])

train['part_fuzzy_orig_title'] = fuzzy_match_partial(train['orig_search_term'], train['orig_product_title'])
train['part_fuzzy_orig_descr'] = fuzzy_match_partial(train['orig_search_term'], train['orig_descr'])

train['first_word_title'] = match_first_word(train['search_term'], train['product_title'])  
train['first_word_descr'] = match_first_word(train['search_term'], train['descr']) 

train['match_numbers_title'] = match_numbers(train['search_term'], train['product_title'])  

train['brand'] = train.product_uid.map(brand['value']).astype(str)
train['brand'] = train['brand'].fillna('').astype(str)
train['fuzzy_brand'] = fuzzy_match_partial(train['brand'], train['search_term'])

CPU times: user 7min 54s, sys: 1.43 s, total: 7min 55s
Wall time: 7min 54s


In [12]:
pd.set_option('display.max_colwidth', -1)
pd.set_option('display.max_columns', None)

In [13]:
train.to_csv('zip/train8.csv.gz', compression='gzip')
train.head()

,product_uid,product_title,search_term,relevance,orig_search_term,orig_product_title,orig_descr,descr,match_product_title,match_descr,len_search_term_words,len_search_term_letters,match_pos_title,match_pos_descr,prod_title_ind,descr_ind,words_std_title,words_std_descr,len_title,len_descr,last_word_title,last_word_descr,match_orig_title,match_orig_descr,fuzzy_title,fuzzy_descr,fuzzy_orig_title,fuzzy_orig_descr,part_fuzzy_title,part_fuzzy_descr,part_fuzzy_orig_title,part_fuzzy_orig_descr,first_word_title,first_word_descr,match_numbers_title,brand,fuzzy_brand
id,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
2,100001,simpson strong tie 12 gaug angl,angl bracket,3.00,angle bracket,Simpson Strong-Tie 12-Gauge Angle,"Not only do angles make joints stronger, they also provide more consistent, straight corners. Simpson Strong-Tie offers a wide variety of angles in various sizes and thicknesses to handle light-duty jobs or projects where a structural connection is needed. Some can be bent (skewed) to match the project. For outdoor projects or those where moisture is present, use our ZMAX zinc-coated connectors, which provide extra resistance against corrosion (look for a ""Z"" at the end of the model number).Versatile connector for various 90 connections and home repair projectsStronger than angled nailing or screw fastening aloneHelp ensure joints are consistently straight and strongDimensions: 3 in. x 3 in. x 1-1/2 in.Made from 12-Gauge steelGalvanized for extra corrosion resistanceInstall with 10d common nails or #9 x 1-1/2 in. Strong-Drive SD screws",angl make joint stronger also provid consist straight corners. simpson strong tie offer wide varieti angl various size thick handl light duti job project structur connect needed. bent skew match project. outdoor project moistur present use zmax zinc coat connector provid extra resist corros look z end model number .versatil connector various 90 connect home repair projectsstrong angl nail screw fasten alonehelp ensur joint consist straight strongdimens 3 in. 3 in. 1 1 2 in. made 12 gaug steelgalvan extra corros resistanceinstal 10d common nail 9 1 1 2 in. strong drive sd screw,0.500000,0.500000,2,12,0.500000,0.000000,0.833333,0.000000,0.00,0.00,6,94,0,0,0.0,0.0,50,50,56,4,100,100,100,62,1,1,0,simpson strong tie,33
3,100001,simpson strong tie 12 gaug angl,bracket,2.50,l bracket,Simpson Strong-Tie 12-Gauge Angle,"Not only do angles make joints stronger, they also provide more consistent, straight corners. Simpson Strong-Tie offers a wide variety of angles in various sizes and thicknesses to handle light-duty jobs or projects where a structural connection is needed. Some can be bent (skewed) to match the project. For outdoor projects or those where moisture is present, use our ZMAX zinc-coated connectors, which provide extra resistance against corrosion (look for a ""Z"" at the end of the model number).Versatile connector for various 90 connections and home repair projectsStronger than angled nailing or screw fastening aloneHelp ensure joints are consistently straight and strongDimensions: 3 in. x 3 in. x 1-1/2 in.Made from 12-Gauge steelGalvanized for extra corrosion resistanceInstall with 10d common nails or #9 x 1-1/2 in. Strong-Drive SD screws",angl make joint stronger also provid consist straight corners. simpson strong tie offer wide varieti angl various size thick handl light duti job project structur connect needed. bent skew match project. outdoor project moistur present use zmax zinc coat connector provid extra resist corros look z end model number .versatil connector various 90 connect home repair projectsstrong angl nail screw fasten alonehelp ensur joint consist straight strongdimens 3 in. 3 in. 1 1 2 in. made 12 gaug steelgalvan extra corros resistanceinstal 10d common nail 9 1 1 2 in. strong drive sd screw,0.000000,0.000000,1,7,0.000000,0.000000,1.000000,1.000000,0.00,0.00,6,94,0,0,0.0,0.0,11,3,19,3,14,43,33,44,0,0,0,simpson strong tie,29
9,100002,behr premium textur deckov 1 gal. sc

Same procedure for the test dataset

In [14]:
test = pd.read_csv('zip/test.csv.gz', index_col=['id'], compression='gzip', encoding=encoding)
test.head()

,product_uid,product_title,search_term
id,,,
1,100001,Simpson Strong-Tie 12-Gauge Angle,90 degree bracket
4,100001,Simpson Strong-Tie 12-Gauge Angle,metal l brackets
5,100001,Simpson Strong-Tie 12-Gauge Angle,simpson sku able
6,100001,Simpson Strong-Tie 12-Gauge Angle,simpson strong ties
7,100001,Simpson Strong-Tie 12-Gauge Angle,simpson strong tie hcc668


In [15]:
test['orig_search_term'] = test['search_term'].astype(str)
test['search_term'] = stem_sentence(test['orig_search_term'].astype(str) )
test.loc[test['search_term'] == '', 'search_term'] = 'null'
test['search_term'] = test['search_term'].astype(str)

test['orig_product_title'] = test['product_title'].astype(str)
test['product_title'] = stem_sentence(test['product_title'].astype(str) )
test['product_title'] = test['product_title'].astype(str)

test['orig_descr'] = test.product_uid.map(products['orig_descr'])
test['descr'] = test.product_uid.map(products['descr'])

test.head()

,product_uid,product_title,search_term,orig_search_term,orig_product_title,orig_descr,descr
id,,,,,,,
1,100001,simpson strong tie 12 gaug angl,90deg. bracket,90 degree bracket,Simpson Strong-Tie 12-Gauge Angle,"Not only do angles make joints stronger, they also provide more consistent, straight corners. Simpson Strong-Tie offers a wide variety of angles in various sizes and thicknesses to handle light-duty jobs or projects where a structural connection is needed. Some can be bent (skewed) to match the project. For outdoor projects or those where moisture is present, use our ZMAX zinc-coated connectors, which provide extra resistance against corrosion (look for a ""Z"" at the end of the model number).Versatile connector for various 90 connections and home repair projectsStronger than angled nailing or screw fastening aloneHelp ensure joints are consistently straight and strongDimensions: 3 in. x 3 in. x 1-1/2 in.Made from 12-Gauge steelGalvanized for extra corrosion resistanceInstall with 10d common nails or #9 x 1-1/2 in. Strong-Drive SD screws",angl make joint stronger also provid consist straight corners. simpson strong tie offer wide varieti angl various size thick handl light duti job project structur connect needed. bent skew match project. outdoor project moistur present use zmax zinc coat connector provid extra resist corros look z end model number .versatil connector various 90 connect home repair projectsstrong angl nail screw fasten alonehelp ensur joint consist straight strongdimens 3 in. 3 in. 1 1 2 in. made 12 gaug steelgalvan extra corros resistanceinstal 10d common nail 9 1 1 2 in. strong drive sd screw
4,100001,simpson strong tie 12 gaug angl,metal bracket,metal l brackets,Simpson Strong-Tie 12-Gauge Angle,"Not only do angles make joints stronger, they also provide more consistent, straight corners. Simpson Strong-Tie offers a wide variety of angles in various sizes and thicknesses to handle light-duty jobs or projects where a structural connection is needed. Some can be bent (skewed) to match the project. For outdoor projects or those where moisture is present, use our ZMAX zinc-coated connectors, which provide extra resistance against corrosion (look for a ""Z"" at the end of the model number).Versatile connector for various 90 connections and home repair projectsStronger than angled nailing or screw fastening aloneHelp ensure joints are consistently straight and strongDimensions: 3 in. x 3 in. x 1-1/2 in.Made from 12-Gauge steelGalvanized for extra corrosion resistanceInstall with 10d common nails or #9 x 1-1/2 in. Strong-Drive SD screws",angl make joint stronger also provid consist straight corners. simpson strong tie offer wide varieti angl various size thick handl light duti job project structur connect needed. bent skew match project. outdoor project moistur present use zmax zinc coat connector provid extra resist corros look z end model number .versatil connector various 90 connect home repair projectsstrong angl nail screw fasten alonehelp ensur joint consist straight strongdimens 3 in. 3 in. 1 1 2 in. made 12 gaug steelgalvan extra corros resistanceinstal 10d common nail 9 1 1 2 in. strong drive sd screw
5,100001,simpson strong tie 12 gaug angl,simpson sku abl,simpson sku able,Simpson Strong-Tie 12-Gauge Angle,"Not only do angles make joints stronger, they also provide more consistent, straight corners. Simpson Strong-Tie offers a wide variety of angles in various sizes and thicknesses to handle light-duty jobs or projects where a structural connection is needed. Some can be bent (skewed) to match the project. For outdoor projects or those where moisture is present, use our ZMAX zinc-coated connectors, which provide extra resistance against corrosion (look for a ""Z"" at the end of the model number).Versatile connector for various 90 connections and home repair projectsStronger than angled nailing or screw fastening aloneHelp ensure joints are consistently straight and strongDimensions: 3 in. x 3 in. 

In [16]:
%%time

for col in ['product_title', 'descr']:
    test['match_' + col] = word_match_count(test['search_term'], test[col])
    
test['len_search_term_words'] = test['search_term'].apply(lambda x: len(x.split()))
test['len_search_term_letters'] = test['search_term'].str.len()

test['match_pos_title'] = match_word_n_pos(test['orig_product_title'], test['orig_search_term'])
test['match_pos_descr'] = match_word_n_pos(test['orig_descr'], test['orig_search_term'])

test['prod_title_ind'] = word_match_index(test['search_term'], test['product_title'])
test['descr_ind'] = word_match_index(test['search_term'], test['descr'])

test['words_std_title'] = word_match_std(test['search_term'], test['product_title'])
test['words_std_descr'] = word_match_std(test['search_term'], test['descr'])

test['len_title'] = test['product_title'].apply(lambda x: len(x.split()))
test['len_descr'] = test['descr'].apply(lambda x: len(x.split()))

test['last_word_title'] = match_last_word(test['search_term'], test['product_title'])  
test['last_word_descr'] = match_last_word(test['search_term'], test['descr']) 

test['match_orig_title'] = word_match_count(test['orig_search_term'], test['orig_product_title'])
test['match_orig_descr'] = word_match_count(test['orig_search_term'], test['orig_descr'])

test['fuzzy_title'] = fuzzy_match(test['search_term'], test['product_title'])
test['fuzzy_descr'] = fuzzy_match(test['search_term'], test['descr'])

test['fuzzy_orig_title'] = fuzzy_match(test['orig_search_term'], test['orig_product_title'])
test['fuzzy_orig_descr'] = fuzzy_match(test['orig_search_term'], test['orig_descr'])

test['part_fuzzy_title'] = fuzzy_match_partial(test['search_term'], test['product_title'])
test['part_fuzzy_descr'] = fuzzy_match_partial(test['search_term'], test['descr'])

test['part_fuzzy_orig_title'] = fuzzy_match_partial(test['orig_search_term'], test['orig_product_title'])
test['part_fuzzy_orig_descr'] = fuzzy_match_partial(test['orig_search_term'], test['orig_descr'])

test['first_word_title'] = match_first_word(test['search_term'], test['product_title'])  
test['first_word_descr'] = match_first_word(test['search_term'], test['descr']) 

test['match_numbers_title'] = match_numbers(test['search_term'], test['product_title'])  

test['brand'] = test.product_uid.map(brand['value']).astype(str)
test['brand'] = test['brand'].fillna('').astype(str)
test['fuzzy_brand'] = fuzzy_match_partial(test['brand'], test['search_term'])

CPU times: user 17min 40s, sys: 4.25 s, total: 17min 44s
Wall time: 17min 43s


In [17]:
test.to_csv('zip/test8.csv.gz', compression='gzip')
test.head()

,product_uid,product_title,search_term,orig_search_term,orig_product_title,orig_descr,descr,match_product_title,match_descr,len_search_term_words,len_search_term_letters,match_pos_title,match_pos_descr,prod_title_ind,descr_ind,words_std_title,words_std_descr,len_title,len_descr,last_word_title,last_word_descr,match_orig_title,match_orig_descr,fuzzy_title,fuzzy_descr,fuzzy_orig_title,fuzzy_orig_descr,part_fuzzy_title,part_fuzzy_descr,part_fuzzy_orig_title,part_fuzzy_orig_descr,first_word_title,first_word_descr,match_numbers_title,brand,fuzzy_brand
id,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
1,100001,simpson strong tie 12 gaug angl,90deg. bracket,90 degree bracket,Simpson Strong-Tie 12-Gauge Angle,"Not only do angles make joints stronger, they also provide more consistent, straight corners. Simpson Strong-Tie offers a wide variety of angles in various sizes and thicknesses to handle light-duty jobs or projects where a structural connection is needed. Some can be bent (skewed) to match the project. For outdoor projects or those where moisture is present, use our ZMAX zinc-coated connectors, which provide extra resistance against corrosion (look for a ""Z"" at the end of the model number).Versatile connector for various 90 connections and home repair projectsStronger than angled nailing or screw fastening aloneHelp ensure joints are consistently straight and strongDimensions: 3 in. x 3 in. x 1-1/2 in.Made from 12-Gauge steelGalvanized for extra corrosion resistanceInstall with 10d common nails or #9 x 1-1/2 in. Strong-Drive SD screws",angl make joint stronger also provid consist straight corners. simpson strong tie offer wide varieti angl various size thick handl light duti job project structur connect needed. bent skew match project. outdoor project moistur present use zmax zinc coat connector provid extra resist corros look z end model number .versatil connector various 90 connect home repair projectsstrong angl nail screw fasten alonehelp ensur joint consist straight strongdimens 3 in. 3 in. 1 1 2 in. made 12 gaug steelgalvan extra corros resistanceinstal 10d common nail 9 1 1 2 in. strong drive sd screw,0.000000,0.000000,2,14,0.000000,0.000000,1.0,1.000000,0,0,6,94,0,0,0.0,0.333333,18,6,28,21,23,38,35,100,0,0,0.0,simpson strong tie,23
4,100001,simpson strong tie 12 gaug angl,metal bracket,metal l brackets,Simpson Strong-Tie 12-Gauge Angle,"Not only do angles make joints stronger, they also provide more consistent, straight corners. Simpson Strong-Tie offers a wide variety of angles in various sizes and thicknesses to handle light-duty jobs or projects where a structural connection is needed. Some can be bent (skewed) to match the project. For outdoor projects or those where moisture is present, use our ZMAX zinc-coated connectors, which provide extra resistance against corrosion (look for a ""Z"" at the end of the model number).Versatile connector for various 90 connections and home repair projectsStronger than angled nailing or screw fastening aloneHelp ensure joints are consistently straight and strongDimensions: 3 in. x 3 in. x 1-1/2 in.Made from 12-Gauge steelGalvanized for extra corrosion resistanceInstall with 10d common nails or #9 x 1-1/2 in. Strong-Drive SD screws",angl make joint stronger also provid consist straight corners. simpson strong tie offer wide varieti angl various size thick handl light duti job project structur connect needed. bent skew match project. outdoor project moistur present use zmax zinc coat connector provid extra resist corros look z end model number .versatil connector various 90 connect home repair projectsstrong angl nail screw fasten alonehelp ensur joint consist straight strongdimens 3 in. 3 in. 1 1 2 in. made 12 gaug steelgalvan extra corros resistanceinstal 10d common nail 9 1 1 2 in. strong drive sd screw,0.000000,0.000000,2,13,0.000000,0.000000,1.0,1.000000,0,0,6,94,0,0,0.0,0.000000,18,6,24,4,23,46,25,50,0,0,0.0,simpson strong tie,23
5,100001,simpson strong tie 12 gaug angl,simpson sku abl

Проверим score на данных

In [18]:
import lightgbm as lgb
from sklearn.model_selection import GridSearchCV
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.decomposition import TruncatedSVD
from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.pipeline import Pipeline, FeatureUnion

class cust_regression_vals(BaseEstimator, TransformerMixin):
    def fit(self, x, y=None):
        return self
    def transform(self, train):
        return train[tree_cols].values


class cust_txt_col(BaseEstimator, TransformerMixin):
    def __init__(self, col):
        self.col = col
    def fit(self, x, y=None):
        return self
    def transform(self, dataset):
        return dataset[self.col].apply(str)

def fitter(tsvd1=80, tsvd2=80, tsvd3=60, tsvd4=30):
    clf = Pipeline([
    ('union', FeatureUnion(
                transformer_list = [
                    ('cst',  cust_regression_vals()),  
                    ('txt1', Pipeline([('s1', cust_txt_col(col='search_term')),
                                       ('tfidf1', TfidfVectorizer(stop_words='english')),
                                       ('tsvd1', TruncatedSVD(n_components=tsvd1))])),
                    ('txt2', Pipeline([('s2', cust_txt_col(col='product_title')),
                                       ('tfidf2', TfidfVectorizer(stop_words='english')),
                                       ('tsvd2', TruncatedSVD(n_components=tsvd2))])),
                    ('txt3', Pipeline([('s3', cust_txt_col(col='descr')),
                                       ('tfidf3', TfidfVectorizer(stop_words='english')),
                                       ('tsvd3', TruncatedSVD(n_components=tsvd3))])),
                    ('txt4', Pipeline([('s4', cust_txt_col(col='brand')),
                                       ('tfidf4', TfidfVectorizer(stop_words='english')),
                                       ('tsvd4', TruncatedSVD(n_components=tsvd4))]))
                    ]
            ))])
    clf.fit(train[use_cols])
    return clf

def write_submission(model, test_data):
    y_pred = model.predict(test_data)
    y_pred[y_pred>3.] = 3.
    y_pred[y_pred<1.] = 1.
    results = pd.DataFrame({'id':test.index.values, 'relevance':y_pred})
    results.to_csv('lgbm5.csv', header=True, index=False)

In [19]:
tree_cols = ['match_product_title', 'match_descr',
             'len_search_term_words', 'len_search_term_letters',
             'match_pos_title', 'match_pos_descr',
             'prod_title_ind', 'descr_ind',
             'words_std_title', 'words_std_descr',
             'len_title', 'len_descr',
             'last_word_title', 'last_word_descr',
             'match_orig_title', 'match_orig_descr',
             'fuzzy_title', 'fuzzy_descr', 'fuzzy_orig_title', 'fuzzy_orig_descr',
             'part_fuzzy_title', 'part_fuzzy_descr', 'part_fuzzy_orig_title', 'part_fuzzy_orig_descr',
             'first_word_title', 'first_word_descr',
             'match_numbers_title',
             'fuzzy_brand']


use_cols = tree_cols + ['search_term', 'product_title', 'descr', 'brand']

In [20]:
y = train['relevance'].values

In [21]:
clf = fitter()
X_train = clf.transform(train[use_cols])
X_test = clf.transform(test[use_cols])

gbm = lgb.LGBMRegressor(learning_rate=0.03, colsample_bytree=0.7, num_leaves=80, n_estimators=1000)
gbm.fit(X_train, y)

tc_len = len(tree_cols)
print(dict(zip(tree_cols, gbm.feature_importances_[:tc_len]) ))

write_submission(gbm, X_test)

{'match_product_title': 296, 'match_descr': 140, 'len_search_term_words': 251, 'len_search_term_letters': 691, 'match_pos_title': 143, 'match_pos_descr': 175, 'prod_title_ind': 681, 'descr_ind': 486, 'words_std_title': 222, 'words_std_descr': 309, 'len_title': 423, 'len_descr': 375, 'last_word_title': 91, 'last_word_descr': 66, 'match_orig_title': 108, 'match_orig_descr': 145, 'fuzzy_title': 576, 'fuzzy_descr': 435, 'fuzzy_orig_title': 598, 'fuzzy_orig_descr': 393, 'part_fuzzy_title': 183, 'part_fuzzy_descr': 105, 'part_fuzzy_orig_title': 178, 'part_fuzzy_orig_descr': 85, 'first_word_title': 80, 'first_word_descr': 53, 'match_numbers_title': 119, 'fuzzy_brand': 308}


0.46838

In [22]:
train['pred_diff'] = train['relevance'] - gbm.predict(X_train)
train.sort_values(['pred_diff'], ascending=False)[:50]

,product_uid,product_title,search_term,relevance,orig_search_term,orig_product_title,orig_descr,descr,match_product_title,match_descr,len_search_term_words,len_search_term_letters,match_pos_title,match_pos_descr,prod_title_ind,descr_ind,words_std_title,words_std_descr,len_title,len_descr,last_word_title,last_word_descr,match_orig_title,match_orig_descr,fuzzy_title,fuzzy_descr,fuzzy_orig_title,fuzzy_orig_descr,part_fuzzy_title,part_fuzzy_descr,part_fuzzy_orig_title,part_fuzzy_orig_descr,first_word_title,first_word_descr,match_numbers_title,brand,fuzzy_brand,pred_diff
id,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
207064,194599,home legend distress lennox hickori 3 8 in. thick 2 1 8 in. wide 78 in. length hardwood carpet reduc mold,lennox,3.00,ennox,Home Legend Distressed Lennox Hickory 3/8 in. Thick x 2-1/8 in. Wide x 78 in. Length Hardwood Carpet Reducer Molding,"78 in. length, hardwood veneer carpet reducer is an overlap molding. It is used to provide a smooth and safe transition between floor coverings of different heights and constructions. It can also be used to as borders around fire place hearth and at sliding door threshold. The prefinished molding can be nailed or glued with its coordinating style of floor.California residents: see&nbsp;Proposition 65 informationWood Species: Hickory3/8 in. Thickness x 2-1/8 in. Width x 78 in. Length MoldingItem coordinates with 3/8 in. thick hardwood flooringConstruction: High Density Fiberboard (HDF) MoldingPrefinished Lennox Hickory, Dark ColorJanka wood hardness rating/resistance to denting: 1820Appropriate Grade for Installation: All GradesInstallation: GlueResidential useAll online orders for this item ship via parcel ground and may arrive in multiple boxes",78 in. length hardwood veneer carpet reduc overlap molding. use provid smooth safe transit floor cover differ height constructions. also use border around fire place hearth slide door threshold. prefinish mold nail glu coordin style floor.california resid see nbsp proposit 65 in. formationwood speci hickory3 8 in. thick 2 1 8 in. width 78 in. length moldingitem coordin 3 8 in. thick hardwood flooringconstruct high densiti fiberboard hdf moldingprefinish lennox hickori dark colorjanka wood hard rate resist dent 1820appropri grade instal gradesinstal glueresidenti useal onlin order item ship via parcel ground may arriv multipl box,1.000000,1.000000,1,6,0.000000,0.000000,0.142857,0.726316,0.000000,0.000000,21,95,1,1,0.000000,0.000000,100,100,9,1,100,100,100,40,1,1,0,home legend,50,1.109605
203914,192088,behr premium plus 600e 2 harbor mist zero voc interior paint,locit 2 plus 1,3.00,locite 2 plus 1,BEHR Premium Plus #600E-2 Harbor Mist Zero VOC Interior Paint,"BEHR Premium Plus Zero VOC, Self-Priming Interior Flat is 100% acrylic and provides a mildew resistant finish. This easy-clean flat provides excellent touch-up, ultimate durability and exceptional hide. The finish displays a non-reflective, matte appearance.California residents: see&nbsp;Proposition 65 informationIdeal for family rooms, living rooms, dining rooms, bedrooms and ceilings100% acrylic finish for easy cleaningTough and durable matte finishExceptional hide, 1 can covers up to 400 sq. ft. depending on color selected and surface porosityEasy soap and water clean-upExceptional latex formula provides a mildew resistant finishSelf priming finish that seals the surfaceActual paint colors may vary from on-screen and printer representationsOnline Price includes Paint Care fee in the following states: CA, CO, CT, ME, MN, OR, RI, VT",behr premium plus zero voc self prime interior flat 100 acryl provid mildew resist finish. easi clean flat provid excel touch ultim durabl except hide. finish display non reflect matt appearance.california resid see nbsp proposit 65 in. formationid famili room live room dine room bedroom ceilings100 acryl finish easi cleaningtough durabl matt finishexcept hide 1 cover 400 sq. ft. depend color select surfac porosityeasi soap water clean upexcept lat

In [23]:
train.sort_values(['pred_diff'], ascending=True)[:50]

,product_uid,product_title,search_term,relevance,orig_search_term,orig_product_title,orig_descr,descr,match_product_title,match_descr,len_search_term_words,len_search_term_letters,match_pos_title,match_pos_descr,prod_title_ind,descr_ind,words_std_title,words_std_descr,len_title,len_descr,last_word_title,last_word_descr,match_orig_title,match_orig_descr,fuzzy_title,fuzzy_descr,fuzzy_orig_title,fuzzy_orig_descr,part_fuzzy_title,part_fuzzy_descr,part_fuzzy_orig_title,part_fuzzy_orig_descr,first_word_title,first_word_descr,match_numbers_title,brand,fuzzy_brand,pred_diff
id,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
69679,119478,romano 4 ft. boxwood spiral topiari tree,topiari tree,1.00,topiary tree,Romano 4 ft. Boxwood Spiral Topiary Tree,"Enhance your home with a Romano Boxwood Spiral Topiary Tree. This wonderful full bodied Boxwood Spiral Topiary Tree keeps its colors when all the rest lose their color and leaves. You can always look forward to the rich green tones of the Romano Boxwood Spiral Topiary Tree in your home. If you are looking for a High quality artificial tree to make a bold statement in your home, this is it.Light and mobile designOver 700 leavesIndoor/outdoorIron frameUV resistant rubber leaves7 in. potEnhances any room in your house",enhanc home romano boxwood spiral topiari tree. wonder full bodi boxwood spiral topiari tree keep color rest lose color leaves. alway look forward rich green tone romano boxwood spiral topiari tree home. look high qualiti artifici tree make bold statement home it.light mobil designov 700 leavesindoor outdooriron frameuv resist rubber leaves7 in. potenh room hous,1.000000,1.000000,2,12,1.000000,1.000000,0.714286,0.090909,0.250000,16.000000,7,55,1,1,0.000000,0.500000,100,100,100,100,100,100,100,100,1,1,0,nan,33,-1.571493
136808,148897,earthquak 212cc tiller rear tine crt side shield,side shield,1.00,side shields,Earthquake 212cc Tiller Rear Tine CRT with Side Shields,"The Earthquake 6015V rear tine rototiller delivers the ultimate combination of power and size. Rear-mounted tines till stubborn dirt easily with a high-performance 212cc Viper engine. We have engineered a compact rear tine rototiller with the same power as its larger counterparts.Easy - single-handed control through sodEarthquake 6015V rear tineCompact - full-sized power in a compact frameDurable - cast iron, bronze gear drive transmission",earthquak 6015v rear tine rototil deliv ultim combin power size. rear mount tine till stubborn dirt easili high perform 212cc viper engine. engin compact rear tine rototil power larger counterparts.easi singl hand control sodearthquak 6015v rear tinecompact full size power compact framedur cast iron bronz gear drive transmiss,1.000000,0.000000,2,11,1.000000,0.000000,0.750000,1.000000,0.250000,0.000000,8,48,1,0,0.000000,0.000000,100,8,100,6,100,64,100,67,1,0,0,nan,0,-1.422191
120624,140844,rachael ray 10 qt. cover stockpot,rachael ray,1.00,rachael ray,Rachael Ray 10 qt. Covered Stockpot,"Whether you're boiling pasta, making a batch of chili or cooking grains, this large, 10 quart stockpot is just the right size. When it's not on the stovetop, its two short side handles help it take up less space in your cabinet. The hard-anodized construction is preferred for its even heat distribution throughout the pan and up the sides. The nonstick cooking surface means you don't need to worry about food sticking or about scrubbing dirty pans - cleanup is a breeze. In fact, Rachael's stockpot is dishwasher safe for added convenience.Hard-anodized cookware is exceptionally durable, plus it heats quickly and evenly, reducing ""hot spots"" that can burn foodsA clear coat exterior makes this cookware dishwasher safe for convenienceLong-lasting nonstick interior lets food slide off with ease and makes cleanup quick and easyFun rubberized handles provide a comfortable grip and are oven safe to 350 degrees FahrenheitTempered glass lid to watch your foods cook without having to remove the lid to keep heat and moi

In [24]:
@np.vectorize
def word_match_std(search_term, text):
    indexes = []
    search_term = search_term.split()
    text = text.split()    
    n = len(text)
    if len(search_term) == 0:
        return 0
    for word in set(search_term):
        if word in text:
            indexes.append(text.index(word))
    if len(indexes) > 0:
        if len(search_term) == 1:
            return 1
        return np.var(indexes)
    else:
        return 0


train['words_std_title'] = word_match_std(train['search_term'], train['product_title'])
train['words_std_descr'] = word_match_std(train['search_term'], train['descr'])

test['words_std_title'] = word_match_std(test['search_term'], test['product_title'])
test['words_std_descr'] = word_match_std(test['search_term'], test['descr'])

X_train = clf.transform(train[use_cols])
X_test = clf.transform(test[use_cols])

gbm = lgb.LGBMRegressor(learning_rate=0.03, colsample_bytree=0.7, num_leaves=80, n_estimators=1000)
gbm.fit(X_train, y)

tc_len = len(tree_cols)
print(dict(zip(tree_cols, gbm.feature_importances_[:tc_len]) ))

write_submission(gbm, X_test)

{'match_product_title': 293, 'match_descr': 159, 'len_search_term_words': 271, 'len_search_term_letters': 714, 'match_pos_title': 159, 'match_pos_descr': 188, 'prod_title_ind': 680, 'descr_ind': 489, 'words_std_title': 231, 'words_std_descr': 280, 'len_title': 446, 'len_descr': 389, 'last_word_title': 98, 'last_word_descr': 81, 'match_orig_title': 115, 'match_orig_descr': 140, 'fuzzy_title': 591, 'fuzzy_descr': 406, 'fuzzy_orig_title': 531, 'fuzzy_orig_descr': 372, 'part_fuzzy_title': 167, 'part_fuzzy_descr': 87, 'part_fuzzy_orig_title': 208, 'part_fuzzy_orig_descr': 79, 'first_word_title': 102, 'first_word_descr': 57, 'match_numbers_title': 109, 'fuzzy_brand': 342}


0.46786

In [25]:
bullets_names = ['Bullet0' + str(i) for i in range(1, 10)]
bullets = attrs[attrs['name'].isin(bullets_names)]
bullets = bullets.groupby(['product_uid'])['value'].apply(lambda x: ' '.join([str(a) for a in x]))
bullets = bullets.apply(stem_sentence)

bullets

product_uid
100001.0    versatil connector various 90 degre connect home repair project stronger angl nail screw fasten alon help ensur joint consist straight strong dimens 3 in. 3 in. 1 1 2 in. made 12 gaug steel galvan extra corros resist instal 10d common nail 9 1 1 2 in. strong drive sd screw                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                        
100002.0    reviv wood composit deck rail porch boat dock also great concret pool deck patio sidewalk 100 acryl solid color coat resis

In [27]:
train['bullets'] = train.product_uid.map(bullets).astype(str)
train['match_bullets'] = word_match_count(train['bullets'], train['search_term'])

test['bullets'] = test.product_uid.map(bullets).astype(str)
test['match_bullets'] = word_match_count(test['bullets'], test['search_term'])

In [28]:
tree_cols = ['match_product_title', 'match_descr',
             'len_search_term_words', 'len_search_term_letters',
             'match_pos_title', 'match_pos_descr',
             'prod_title_ind', 'descr_ind',
             'words_std_title', 'words_std_descr',
             'len_title', 'len_descr',
             'last_word_title', 'last_word_descr',
             'match_orig_title', 'match_orig_descr',
             'fuzzy_title', 'fuzzy_descr', 'fuzzy_orig_title', 'fuzzy_orig_descr',
             'part_fuzzy_title', 'part_fuzzy_descr', 'part_fuzzy_orig_title', 'part_fuzzy_orig_descr',
             'first_word_title', 'first_word_descr',
             'match_numbers_title',
             'fuzzy_brand',
             'match_bullets']


use_cols = tree_cols + ['search_term', 'product_title', 'descr', 'brand']

clf = fitter()
X_train = clf.transform(train[use_cols])
X_test = clf.transform(test[use_cols])

gbm = lgb.LGBMRegressor(learning_rate=0.03, colsample_bytree=0.7, num_leaves=80, n_estimators=1000)
gbm.fit(X_train, y)

tc_len = len(tree_cols)
print(dict(zip(tree_cols, gbm.feature_importances_[:tc_len]) ))

write_submission(gbm, X_test)

{'match_product_title': 297, 'match_descr': 137, 'len_search_term_words': 256, 'len_search_term_letters': 676, 'match_pos_title': 163, 'match_pos_descr': 177, 'prod_title_ind': 662, 'descr_ind': 430, 'words_std_title': 219, 'words_std_descr': 296, 'len_title': 415, 'len_descr': 345, 'last_word_title': 102, 'last_word_descr': 86, 'match_orig_title': 118, 'match_orig_descr': 160, 'fuzzy_title': 551, 'fuzzy_descr': 435, 'fuzzy_orig_title': 565, 'fuzzy_orig_descr': 387, 'part_fuzzy_title': 189, 'part_fuzzy_descr': 84, 'part_fuzzy_orig_title': 205, 'part_fuzzy_orig_descr': 82, 'first_word_title': 92, 'first_word_descr': 50, 'match_numbers_title': 115, 'fuzzy_brand': 317, 'match_bullets': 257}


0.46752

In [30]:
def fitter5(tsvd1=80, tsvd2=80, tsvd3=60, tsvd4=30, tsvd5=30):
    clf = Pipeline([
    ('union', FeatureUnion(
                transformer_list = [
                    ('cst',  cust_regression_vals()),  
                    ('txt1', Pipeline([('s1', cust_txt_col(col='search_term')),
                                       ('tfidf1', TfidfVectorizer(stop_words='english')),
                                       ('tsvd1', TruncatedSVD(n_components=tsvd1))])),
                    ('txt2', Pipeline([('s2', cust_txt_col(col='product_title')),
                                       ('tfidf2', TfidfVectorizer(stop_words='english')),
                                       ('tsvd2', TruncatedSVD(n_components=tsvd2))])),
                    ('txt3', Pipeline([('s3', cust_txt_col(col='descr')),
                                       ('tfidf3', TfidfVectorizer(stop_words='english')),
                                       ('tsvd3', TruncatedSVD(n_components=tsvd3))])),
                    ('txt4', Pipeline([('s4', cust_txt_col(col='brand')),
                                       ('tfidf4', TfidfVectorizer(stop_words='english')),
                                       ('tsvd4', TruncatedSVD(n_components=tsvd4))])),
                    ('txt5', Pipeline([('s5', cust_txt_col(col='bullets')),
                                       ('tfidf5', TfidfVectorizer(stop_words='english')),
                                       ('tsvd5', TruncatedSVD(n_components=tsvd5))]))
                    ]
            ))])
    clf.fit(train[use_cols])
    return clf

tree_cols = ['match_product_title', 'match_descr',
             'len_search_term_words', 'len_search_term_letters',
             'match_pos_title', 'match_pos_descr',
             'prod_title_ind', 'descr_ind',
             'words_std_title', 'words_std_descr',
             'len_title', 'len_descr',
             'last_word_title', 'last_word_descr',
             'match_orig_title', 'match_orig_descr',
             'fuzzy_title', 'fuzzy_descr', 'fuzzy_orig_title', 'fuzzy_orig_descr',
             'part_fuzzy_title', 'part_fuzzy_descr', 'part_fuzzy_orig_title', 'part_fuzzy_orig_descr',
             'first_word_title', 'first_word_descr',
             'match_numbers_title',
             'fuzzy_brand',
             'match_bullets']


use_cols = tree_cols + ['search_term', 'product_title', 'descr', 'brand', 'bullets']

clf = fitter()
X_train = clf.transform(train[use_cols])
X_test = clf.transform(test[use_cols])

gbm = lgb.LGBMRegressor(learning_rate=0.03, colsample_bytree=0.7, num_leaves=80, n_estimators=1000)
gbm.fit(X_train, y)

tc_len = len(tree_cols)
print(dict(zip(tree_cols, gbm.feature_importances_[:tc_len]) ))
print(gbm.feature_importances_[tc_len:])

write_submission(gbm, X_test)

{'match_product_title': 290, 'match_descr': 147, 'len_search_term_words': 264, 'len_search_term_letters': 676, 'match_pos_title': 182, 'match_pos_descr': 158, 'prod_title_ind': 656, 'descr_ind': 444, 'words_std_title': 194, 'words_std_descr': 294, 'len_title': 425, 'len_descr': 371, 'last_word_title': 89, 'last_word_descr': 84, 'match_orig_title': 130, 'match_orig_descr': 165, 'fuzzy_title': 545, 'fuzzy_descr': 426, 'fuzzy_orig_title': 561, 'fuzzy_orig_descr': 368, 'part_fuzzy_title': 179, 'part_fuzzy_descr': 89, 'part_fuzzy_orig_title': 200, 'part_fuzzy_orig_descr': 87, 'first_word_title': 75, 'first_word_descr': 41, 'match_numbers_title': 118, 'fuzzy_brand': 326, 'match_bullets': 244}
[495 336 350 306 343 276 274 243 257 307 306 236 259 324 321 261 290 259
 246 300 323 326 304 288 312 300 306 263 271 298 270 274 286 375 297 282
 251 292 320 313 248 318 342 299 297 309 267 308 322 341 319 305 288 341
 303 271 342 329 303 299 341 371 322 341 320 295 342 356 318 389 318 305
 336 335 311

0.46747

In [31]:
train['pred_diff'] = train['relevance'] - gbm.predict(X_train)
train.sort_values(['pred_diff'], ascending=False)[:50]

,product_uid,product_title,search_term,relevance,orig_search_term,orig_product_title,orig_descr,descr,match_product_title,match_descr,len_search_term_words,len_search_term_letters,match_pos_title,match_pos_descr,prod_title_ind,descr_ind,words_std_title,words_std_descr,len_title,len_descr,last_word_title,last_word_descr,match_orig_title,match_orig_descr,fuzzy_title,fuzzy_descr,fuzzy_orig_title,fuzzy_orig_descr,part_fuzzy_title,part_fuzzy_descr,part_fuzzy_orig_title,part_fuzzy_orig_descr,first_word_title,first_word_descr,match_numbers_title,brand,fuzzy_brand,pred_diff,bullets,match_bullets
id,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
203914,192088,behr premium plus 600e 2 harbor mist zero voc interior paint,locit 2 plus 1,3.00,locite 2 plus 1,BEHR Premium Plus #600E-2 Harbor Mist Zero VOC Interior Paint,"BEHR Premium Plus Zero VOC, Self-Priming Interior Flat is 100% acrylic and provides a mildew resistant finish. This easy-clean flat provides excellent touch-up, ultimate durability and exceptional hide. The finish displays a non-reflective, matte appearance.California residents: see&nbsp;Proposition 65 informationIdeal for family rooms, living rooms, dining rooms, bedrooms and ceilings100% acrylic finish for easy cleaningTough and durable matte finishExceptional hide, 1 can covers up to 400 sq. ft. depending on color selected and surface porosityEasy soap and water clean-upExceptional latex formula provides a mildew resistant finishSelf priming finish that seals the surfaceActual paint colors may vary from on-screen and printer representationsOnline Price includes Paint Care fee in the following states: CA, CO, CT, ME, MN, OR, RI, VT",behr premium plus zero voc self prime interior flat 100 acryl provid mildew resist finish. easi clean flat provid excel touch ultim durabl except hide. finish display non reflect matt appearance.california resid see nbsp proposit 65 in. formationid famili room live room dine room bedroom ceilings100 acryl finish easi cleaningtough durabl matt finishexcept hide 1 cover 400 sq. ft. depend color select surfac porosityeasi soap water clean upexcept latex formula provid mildew resist finishself prime finish seal surfaceactu paint color may vari screen printer representationsonlin price includ paint care fee follow state ca co ct mn ri vt,0.500000,0.500000,4,14,0.500000,0.500000,0.181818,0.020408,1.000000,676.00,11,98,0,1,0.000000,0.250000,60,60,57,57,100,100,100,100,0,0,0,behr premium plus,100,1.135760,ideal famili room live room dine room bedroom ceil 100 acryl finish easi clean tough durabl matt finish except hide 1 cover 400 sq. ft. depend color select surfac poros easi soap water clean except latex formula provid mildew resist finish self prime finish seal surfac actual paint color may vari screen printer represent,0.018519
194147,184559,lenmar nickel metal hydrid 1200mah 3.6 volt cordless phone replac batteri,polish batteri metal,3.00,polisher battery metal,Lenmar Nickel-Metal Hydride 1200mAh/3.6-Volt Cordless Phone Replacement Battery,"Power your Panasonic, American Telecom, Tele-Phone, Key Phone, Audiovox, Uniden, Universal, and Zeta cordless phones with Lenmar's Nickel-Metal Hydride cordless phone replacement battery. This 3.6-Volt/1200mAh battery will provide reliable, long-lasting power. Compatible with Panasonic KX-A150, American Telecom 2110, Tele-Phone TEL-1000, Key Phone 920-WD, Audiovox AT-12A, Uniden EX-3300, Universal 2000, and Zeta 510.Fits: Panasonic KX-A150, American Telecom 2110, Tele-Phone TEL-1000, Key Phone 920-WD, Audiovox AT-12A, Uniden EX-3300, Universal 2000, Zeta 510Replaces: Dantona 3AA, Empire CPB-400, General Electric BT-17, Panasonic P-P341PA, Recoton T121, Uniden BA-3003.6-volt / 1200mAhNickel-metal hydride1 year warranty",power panason american telecom tele phone key phone audiovox uniden univers zeta cordless phone lenmar nickel metal hydrid cordless phone replac battery. 3.6 volt 1200mah batteri provid reliabl long last power. compat panason kx a150 american telecom 21

In [32]:
train.sort_values(['pred_diff'], ascending=True)[:50]

,product_uid,product_title,search_term,relevance,orig_search_term,orig_product_title,orig_descr,descr,match_product_title,match_descr,len_search_term_words,len_search_term_letters,match_pos_title,match_pos_descr,prod_title_ind,descr_ind,words_std_title,words_std_descr,len_title,len_descr,last_word_title,last_word_descr,match_orig_title,match_orig_descr,fuzzy_title,fuzzy_descr,fuzzy_orig_title,fuzzy_orig_descr,part_fuzzy_title,part_fuzzy_descr,part_fuzzy_orig_title,part_fuzzy_orig_descr,first_word_title,first_word_descr,match_numbers_title,brand,fuzzy_brand,pred_diff,bullets,match_bullets
id,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
69679,119478,romano 4 ft. boxwood spiral topiari tree,topiari tree,1.00,topiary tree,Romano 4 ft. Boxwood Spiral Topiary Tree,"Enhance your home with a Romano Boxwood Spiral Topiary Tree. This wonderful full bodied Boxwood Spiral Topiary Tree keeps its colors when all the rest lose their color and leaves. You can always look forward to the rich green tones of the Romano Boxwood Spiral Topiary Tree in your home. If you are looking for a High quality artificial tree to make a bold statement in your home, this is it.Light and mobile designOver 700 leavesIndoor/outdoorIron frameUV resistant rubber leaves7 in. potEnhances any room in your house",enhanc home romano boxwood spiral topiari tree. wonder full bodi boxwood spiral topiari tree keep color rest lose color leaves. alway look forward rich green tone romano boxwood spiral topiari tree home. look high qualiti artifici tree make bold statement home it.light mobil designov 700 leavesindoor outdooriron frameuv resist rubber leaves7 in. potenh room hous,1.000000,1.000000,2,12,1.000000,1.000000,0.714286,0.090909,0.250000,16.000000,7,55,1,1,0.000000,0.500000,100,100,100,100,100,100,100,100,1,1,0,nan,33,-1.607430,nan,0.000000
136808,148897,earthquak 212cc tiller rear tine crt side shield,side shield,1.00,side shields,Earthquake 212cc Tiller Rear Tine CRT with Side Shields,"The Earthquake 6015V rear tine rototiller delivers the ultimate combination of power and size. Rear-mounted tines till stubborn dirt easily with a high-performance 212cc Viper engine. We have engineered a compact rear tine rototiller with the same power as its larger counterparts.Easy - single-handed control through sodEarthquake 6015V rear tineCompact - full-sized power in a compact frameDurable - cast iron, bronze gear drive transmission",earthquak 6015v rear tine rototil deliv ultim combin power size. rear mount tine till stubborn dirt easili high perform 212cc viper engine. engin compact rear tine rototil power larger counterparts.easi singl hand control sodearthquak 6015v rear tinecompact full size power compact framedur cast iron bronz gear drive transmiss,1.000000,0.000000,2,11,1.000000,0.000000,0.750000,1.000000,0.250000,0.000000,8,48,1,0,0.000000,0.000000,100,8,100,6,100,64,100,67,1,0,0,nan,0,-1.438943,nan,0.000000
120624,140844,rachael ray 10 qt. cover stockpot,rachael ray,1.00,rachael ray,Rachael Ray 10 qt. Covered Stockpot,"Whether you're boiling pasta, making a batch of chili or cooking grains, this large, 10 quart stockpot is just the right size. When it's not on the stovetop, its two short side handles help it take up less space in your cabinet. The hard-anodized construction is preferred for its even heat distribution throughout the pan and up the sides. The nonstick cooking surface means you don't need to worry about food sticking or about scrubbing dirty pans - cleanup is a breeze. In fact, Rachael's stockpot is dishwasher safe for added convenience.Hard-anodized cookware is exceptionally durable, plus it heats quickly and evenly, reducing ""hot spots"" that can burn foodsA clear coat exterior makes this cookware dishwasher safe for convenienceLong-lasting nonstick interior lets food slide off with ease and makes cleanup quick and easyFun rubberized handles provide a comfortable grip and are oven safe to 350 degrees FahrenheitTempered glass lid to watch your foods cook wit

In [34]:
train['look_part'] = train['search_term'].str.contains('(part|case|cover|tool|kit)')
train['is_part'] = train['product_title'].str.contains('(part|case|cover|tool|kit)')

test['look_part'] = test['search_term'].str.contains('(part|case|cover|tool|kit)')
test['is_part'] = test['product_title'].str.contains('(part|case|cover|tool|kit)')

tree_cols = ['match_product_title', 'match_descr',
             'len_search_term_words', 'len_search_term_letters',
             'match_pos_title', 'match_pos_descr',
             'prod_title_ind', 'descr_ind',
             'words_std_title', 'words_std_descr',
             'len_title', 'len_descr',
             'last_word_title', 'last_word_descr',
             'match_orig_title', 'match_orig_descr',
             'fuzzy_title', 'fuzzy_descr', 'fuzzy_orig_title', 'fuzzy_orig_descr',
             'part_fuzzy_title', 'part_fuzzy_descr', 'part_fuzzy_orig_title', 'part_fuzzy_orig_descr',
             'first_word_title', 'first_word_descr',
             'match_numbers_title',
             'fuzzy_brand',
             'match_bullets',
             'look_part', 'is_part']

use_cols = tree_cols + ['search_term', 'product_title', 'descr', 'brand', 'bullets']

clf = fitter5(tsvd5=20)
X_train = clf.transform(train[use_cols])
X_test = clf.transform(test[use_cols])

gbm = lgb.LGBMRegressor(learning_rate=0.03, colsample_bytree=0.5, num_leaves=80, n_estimators=1000)
gbm.fit(X_train, y)

tc_len = len(tree_cols)
print(dict(zip(tree_cols, gbm.feature_importances_[:tc_len]) ))
print(gbm.feature_importances_[tc_len:])

write_submission(gbm, X_test)

/home/eugene/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:1: UserWarning: This pattern has match groups. To actually get the groups, use str.extract.
  """Entry point for launching an IPython kernel.
/home/eugene/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:2: UserWarning: This pattern has match groups. To actually get the groups, use str.extract.
  
/home/eugene/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:4: UserWarning: This pattern has match groups. To actually get the groups, use str.extract.
  after removing the cwd from sys.path.
/home/eugene/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:5: UserWarning: This pattern has match groups. To actually get the groups, use str.extract.
  """


{'match_product_title': 282, 'match_descr': 130, 'len_search_term_words': 215, 'len_search_term_letters': 623, 'match_pos_title': 142, 'match_pos_descr': 171, 'prod_title_ind': 574, 'descr_ind': 377, 'words_std_title': 209, 'words_std_descr': 272, 'len_title': 365, 'len_descr': 346, 'last_word_title': 74, 'last_word_descr': 64, 'match_orig_title': 96, 'match_orig_descr': 159, 'fuzzy_title': 521, 'fuzzy_descr': 414, 'fuzzy_orig_title': 527, 'fuzzy_orig_descr': 364, 'part_fuzzy_title': 188, 'part_fuzzy_descr': 81, 'part_fuzzy_orig_title': 203, 'part_fuzzy_orig_descr': 69, 'first_word_title': 76, 'first_word_descr': 35, 'match_numbers_title': 110, 'fuzzy_brand': 293, 'match_bullets': 202, 'look_part': 11, 'is_part': 44}
[375 347 272 266 325 273 253 305 248 311 305 294 278 332 290 273 266 288
 273 352 262 309 238 284 306 272 283 278 251 302 255 302 279 339 277 283
 282 331 278 281 283 256 266 291 266 310 274 270 304 275 324 271 273 295
 353 296 300 319 322 330 241 375 305 334 291 345 326 2

0.46728

In [35]:
gbm = lgb.LGBMRegressor(learning_rate=0.02, colsample_bytree=0.7, num_leaves=100, n_estimators=1000)
gbm.fit(X_train, y)

tc_len = len(tree_cols)
print(dict(zip(tree_cols, gbm.feature_importances_[:tc_len]) ))
print(gbm.feature_importances_[tc_len:])

write_submission(gbm, X_test)

{'match_product_title': 390, 'match_descr': 184, 'len_search_term_words': 356, 'len_search_term_letters': 915, 'match_pos_title': 190, 'match_pos_descr': 205, 'prod_title_ind': 859, 'descr_ind': 559, 'words_std_title': 283, 'words_std_descr': 375, 'len_title': 520, 'len_descr': 423, 'last_word_title': 118, 'last_word_descr': 102, 'match_orig_title': 171, 'match_orig_descr': 170, 'fuzzy_title': 681, 'fuzzy_descr': 505, 'fuzzy_orig_title': 728, 'fuzzy_orig_descr': 428, 'part_fuzzy_title': 244, 'part_fuzzy_descr': 124, 'part_fuzzy_orig_title': 225, 'part_fuzzy_orig_descr': 90, 'first_word_title': 117, 'first_word_descr': 59, 'match_numbers_title': 160, 'fuzzy_brand': 417, 'match_bullets': 268, 'look_part': 5, 'is_part': 65}
[520 466 380 338 417 300 358 349 292 362 350 312 355 456 419 306 321 349
 370 384 354 360 333 355 310 370 363 346 303 412 330 342 363 414 332 369
 349 412 404 346 314 337 414 339 345 411 336 351 340 316 400 364 337 372
 431 328 352 427 444 364 364 450 351 351 387 405 4

0.46654

In [45]:
tree_cols = ['match_product_title', 'match_descr',
             'len_search_term_words', 'len_search_term_letters',
             'match_pos_title', 'match_pos_descr',
             'prod_title_ind', 'descr_ind',
             'words_std_title', 'words_std_descr',
             'len_title', 'len_descr',
             'last_word_title', 'last_word_descr',
             'match_orig_title', 'match_orig_descr',
             'fuzzy_title', 'fuzzy_descr', 'fuzzy_orig_title', 'fuzzy_orig_descr',
             'part_fuzzy_title', 'part_fuzzy_descr', 'part_fuzzy_orig_title', 'part_fuzzy_orig_descr',
             'first_word_title', 'first_word_descr',
             'match_numbers_title',
             'fuzzy_brand',
             'match_bullets',
             'look_part', 'is_part']

use_cols = tree_cols + ['search_term', 'product_title', 'descr', 'brand', 'bullets']

clf = fitter5(tsvd4=20, tsvd5=20)
X_train = clf.transform(train[use_cols])
X_test = clf.transform(test[use_cols])

gbm = lgb.LGBMRegressor(learning_rate=0.02, colsample_bytree=0.7, num_leaves=100, n_estimators=1000)
gbm.fit(X_train, y)

tc_len = len(tree_cols)
print(dict(zip(tree_cols, gbm.feature_importances_[:tc_len]) ))
print(gbm.feature_importances_[tc_len:])

write_submission(gbm, X_test)

{'match_product_title': 387, 'match_descr': 217, 'len_search_term_words': 375, 'len_search_term_letters': 926, 'match_pos_title': 171, 'match_pos_descr': 200, 'prod_title_ind': 856, 'descr_ind': 544, 'words_std_title': 275, 'words_std_descr': 362, 'len_title': 548, 'len_descr': 359, 'last_word_title': 122, 'last_word_descr': 103, 'match_orig_title': 165, 'match_orig_descr': 161, 'fuzzy_title': 686, 'fuzzy_descr': 449, 'fuzzy_orig_title': 751, 'fuzzy_orig_descr': 427, 'part_fuzzy_title': 239, 'part_fuzzy_descr': 102, 'part_fuzzy_orig_title': 245, 'part_fuzzy_orig_descr': 69, 'first_word_title': 131, 'first_word_descr': 61, 'match_numbers_title': 169, 'fuzzy_brand': 430, 'match_bullets': 252, 'look_part': 11, 'is_part': 72}
[543 426 357 343 421 356 380 353 272 331 356 304 362 453 381 325 365 373
 308 423 366 351 276 370 392 359 351 299 336 372 341 331 350 462 351 338
 386 331 341 358 409 372 373 364 360 363 339 351 381 363 379 343 317 379
 389 415 339 406 391 377 340 468 345 412 361 386 

In [44]:
train.to_csv('zip/train8.csv.gz', compression='gzip')
test.to_csv('zip/test8.csv.gz', compression='gzip')